In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/climate-dataset-84k/export.csv


# Cell 1: Dependencies

In [2]:
!pip -q install -U transformers datasets accelerate scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 79.2 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 120.9 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.4 requires fastcore<1.9,>=1.8.0, but you have fastcore 1.11.3 which is incompatible.


In [3]:
!pip install iterative-stratification

# Cell 2: Import Libraries

In [4]:
import os
import re
import gc
import math
import random
import numpy as np
import pandas as pd

import torch
from torch import nn

from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    DataCollatorWithPadding,
    set_seed
)

SEED = 42
set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


2026-01-25 17:29:08.818648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769362149.023735      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769362149.090346      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769362149.600184      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769362149.600229      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769362149.600232      55 computation_placer.cc:177] computation placer alr

Device: cuda


# Cell 3: Load dataset

In [7]:
CSV_PATH = "/kaggle/input/climate-dataset-84k/export.csv"  # <-- change this

df = pd.read_csv(CSV_PATH)
print(df.shape)
df

(78401, 38)


,title,publication_year,doi,institutions,idx,authors,openalex_id,abstract,meth|0,reg|0,...,gov|0,ins|3,meth|1,econ|2,lvl|0,sec|5,sec|1,sec|2,ins|4,sec|3
0,Modelling Stand Dynamics after Selective Loggi...,2012.0,NaN,NaN,0,NaN,W999667810,Abstract: Forest degradation and biomass damag...,0.990234,0.020004,...,0.010002,0.020004,0.000000,0.010002,1.000000,0.010002,0.020004,0.010002,0.000000,0.029999
1,"Mathematics, Politics, and Greenhouse Gas Inte...",2003.0,10.5951/mt.96.9.0646,"California State University, Dominguez Hills",1,Stan Yoshinobu,W999644455,While reading the newspaper one morning in spr...,0.640137,0.010002,...,0.000000,0.919922,0.239990,0.010002,0.010002,0.010002,0.010002,0.010002,0.010002,0.020004
2,The Contribution of Rural Development Programm...,2012.0,10.1007/978-3-642-22266-5_23,Istituto Superiore per la Protezione e la Rice...,2,Rocío Dánica Cóndor; Marina Vitullo; Domenico ...,W999485533,"The Health Check reform, reinforcing the Commo...",0.620117,0.119995,...,0.049988,0.959961,0.239990,0.130005,0.010002,0.020004,0.020004,0.010002,0.000000,0.059998
3,Negotiating greenhouse abatement and the theor...,1999.0,10.1007/978-94-015-9169-0_3,Australian Bureau of Agricultural and Resource...,3,Mike Hinchy; Brian S. Fisher,W998914803,The initial qualified commitment of Annex 1 co...,0.529785,0.020004,...,0.020004,0.310059,0.320068,0.010002,1.000000,0.010002,0.010002,0.010002,0.990234,0.020004
4,Is the introduction of a carbon tax a ‘teachab...,2012.0,NaN,NaN,4,Liam Smith; Bas Verplanken; Jim Curtis,W998247206,With the arrival of the carbon tax earlier thi...,0.439941,0.010002,...,0.000000,0.010002,0.439941,0.020004,0.010002,0.010002,0.020004,0.010002,0.010002,0.109985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78396,Carbon Neutrality as Leverage in Transitioning...,2006.0,NaN,NaN,78396,Melanie Dubin; Magdalena Szpala; Tamara Connell,W1001986490,Climate change is one of the most pressing env...,0.040009,0.020004,...,0.010002,0.010002,0.930176,0.000000,1.000000,0.010002,0.000000,0.040009,0.010002,0.059998
78397,Fiscal mechanisms to promote CO2 for enhanced ...,2005.0,10.1016/b978-008044704-9/50188-9,CO2-Norway (Norway),78397,David L. Coleman; Carl-W. Hustad; J. Michael A...,W100136754,The use of carbon dioxide (CO2) for enhanced o...,0.020004,0.080017,...,0.020004,0.130005,0.970215,0.020004,0.959961,0.010002,0.010002,0.229980,0.180054,0.970215
78398,Experimental comparison between R152a and R134...,2015.0,10.1016/j.ijrefrig.2015.06.021,Universitat Politècnica de València; Jaume I U...,78398,Rodrigo Llopis; R. Cabello; E. Torrella; Danie...,W1001294588,Abstract The EU Regulation 517/2014 has recent...,0.939941,0.029999,...,0.020004,0.090027,0.020004,0.029999,1.000000,0.010002,0.229980,0.020004,0.049988,0.150024
78399,BRIEF COMMUNICATION: Genetic control of the ru...,2015.0,NaN,AgResearch,78399,Pinares-Patino Cs; Wood Gr; Hickey Sm; Kirk Mr...,W1000733675,Introduction Methane is a waste product genera...,0.750000,0.020004,...,0.020004,0.059998,0.140015,0.020004,0.250000,0.020004,0.020004,0.010002,0.010002,0.040009


# Cell 4 — Identify label columns automatically

This detects all columns like econ|0, reg|2, sec|6, etc.

In [9]:
meta_cols = {"title", "publication_year", "doi", "institutions", "idx", "authors", "openalex_id", "abstract"}
label_cols = [c for c in df.columns if c not in meta_cols and "|" in c]

# keep only columns that look like category|digit
label_cols = [c for c in label_cols if re.search(r"\|\d+$", c)]
label_cols = sorted(label_cols)

print("Num label columns:", len(label_cols))
print(label_cols[:20])


Num label columns: 30
['econ|0', 'econ|1', 'econ|2', 'econ|3', 'edu|0', 'ev|0', 'ev|1', 'gov|0', 'gov|1', 'gov|2', 'ins|0', 'ins|1', 'ins|2', 'ins|3', 'ins|4', 'lvl|0', 'lvl|1', 'lvl|2', 'meth|0', 'meth|1']


# Cell 5 — Subsample 20,000 rows + make text field

In [10]:
# Keep only rows that have some text
df["title"] = df["title"].fillna("").astype(str)
df["abstract"] = df["abstract"].fillna("").astype(str)

# Simple abstract cleaning (remove extra whitespace)
def clean_text(s):
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["title"] = df["title"].apply(clean_text)
df["abstract"] = df["abstract"].apply(clean_text)

# Combine title + abstract
df["text"] = (df["title"] + " [SEP] " + df["abstract"]).str.strip()

# Ensure labels are numeric 0/1
for c in label_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int).clip(0, 1)

# Remove empty text rows
df = df[df["text"].str.len() > 5].reset_index(drop=True)

# Subsample
N = 20000
if len(df) > N:
    df = df.sample(n=N, random_state=SEED).reset_index(drop=True)

print("After subsample:", df.shape)
df[["text"] + label_cols[:5]].head()


After subsample: (20000, 39)


,text,econ|0,econ|1,econ|2,econ|3,edu|0
0,Does urban vegetation enhance carbon sequestra...,0,0,0,0,0
1,Exploring the driving forces of distributed en...,0,0,0,0,0
2,Energy consumption analysis for CO2 separation...,0,0,0,0,0
3,Economic and employment effects of China's pow...,0,0,0,0,0
4,Leveraging Carbon Markets for Green Buildings:...,0,0,0,0,0


# Cell 6 — Multi-label stratified split (train/val/test)

In [11]:
Y = df[label_cols].values

msss1 = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
train_idx, temp_idx = next(msss1.split(df, Y))

df_train = df.iloc[train_idx].reset_index(drop=True)
df_temp  = df.iloc[temp_idx].reset_index(drop=True)

Y_temp = df_temp[label_cols].values
msss2 = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=SEED)
val_idx, test_idx = next(msss2.split(df_temp, Y_temp))

df_val  = df_temp.iloc[val_idx].reset_index(drop=True)
df_test = df_temp.iloc[test_idx].reset_index(drop=True)

print("Train:", df_train.shape, "Val:", df_val.shape, "Test:", df_test.shape)

# sanity: label prevalence
print("Avg labels per sample (train):", df_train[label_cols].sum(axis=1).mean())
print("Avg labels per sample (val):", df_val[label_cols].sum(axis=1).mean())
print("Avg labels per sample (test):", df_test[label_cols].sum(axis=1).mean())

Train: (16000, 39) Val: (2000, 39) Test: (2000, 39)
Avg labels per sample (train): 0.8450625
Avg labels per sample (val): 0.843
Avg labels per sample (test): 0.8465


# Cell 7 — Baseline model (TF-IDF + One-vs-Rest Logistic Regression)

In [12]:
X_train = df_train["text"].tolist()
X_val   = df_val["text"].tolist()
X_test  = df_test["text"].tolist()

y_train = df_train[label_cols].values
y_val   = df_val[label_cols].values
y_test  = df_test[label_cols].values

tfidf = TfidfVectorizer(max_features=100000, ngram_range=(1,2), min_df=2)
Xtr = tfidf.fit_transform(X_train)
Xva = tfidf.transform(X_val)
Xte = tfidf.transform(X_test)

clf = OneVsRestClassifier(LogisticRegression(max_iter=200, n_jobs=-1))
clf.fit(Xtr, y_train)

val_pred = (clf.predict_proba(Xva) >= 0.5).astype(int)
print("Baseline TF-IDF OVR Logistic")
print("Macro-F1:", f1_score(y_val, val_pred, average="macro", zero_division=0))
print("Micro-F1:", f1_score(y_val, val_pred, average="micro", zero_division=0))

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 2 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 3 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 4 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/mul

Baseline TF-IDF OVR Logistic
Macro-F1: 0.062358516057566295
Micro-F1: 0.5128205128205128


/usr/local/lib/python3.12/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 27 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 28 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 29 is present in all training examples.
  warnings.warn(


# Cell 8 — Tokenizer & Model

In [13]:
MODEL_CANDIDATES = [
    "climatebert/distilroberta-base-climate-f",
    "climatebert/distilroberta-base-climate-detection",
    "distilroberta-base",  # fallback if ClimateBERT name differs on Kaggle
]

MODEL_NAME = None
tokenizer = None
model = None

for cand in MODEL_CANDIDATES:
    try:
        tokenizer = AutoTokenizer.from_pretrained(cand, use_fast=True)
        model = AutoModelForSequenceClassification.from_pretrained(
            cand,
            num_labels=len(label_cols),
            problem_type="multi_label_classification"
        )
        MODEL_NAME = cand
        break
    except Exception as e:
        print("Failed:", cand, "|", str(e)[:120])

print("Using model:", MODEL_NAME)
model.to(device);

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using model: climatebert/distilroberta-base-climate-f


# Cell 9 — Build HuggingFace Datasets

In [14]:
train_ds = Dataset.from_pandas(df_train[["text"] + label_cols])
val_ds   = Dataset.from_pandas(df_val[["text"] + label_cols])
test_ds  = Dataset.from_pandas(df_test[["text"] + label_cols])

def tokenize_batch(batch):
    return tokenizer(batch["text"], truncation=True, max_length=256)

train_ds = train_ds.map(tokenize_batch, batched=True, remove_columns=["text"])
val_ds   = val_ds.map(tokenize_batch, batched=True, remove_columns=["text"])
test_ds  = test_ds.map(tokenize_batch, batched=True, remove_columns=["text"])

# Transformers expects labels in a "labels" field
def to_labels(batch):
    batch["labels"] = [ [batch[c][i] for c in label_cols] for i in range(len(batch[label_cols[0]])) ]
    for c in label_cols:
        del batch[c]
    return batch

train_ds = train_ds.map(to_labels, batched=True)
val_ds   = val_ds.map(to_labels, batched=True)
test_ds  = test_ds.map(to_labels, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

# Cell 10 — Class imbalance handling (pos_weight)

In [15]:
y_train = df_train[label_cols].values
pos = y_train.sum(axis=0)
neg = y_train.shape[0] - pos
pos_weight = (neg / (pos + 1e-6)).astype(np.float32)
pos_weight = np.clip(pos_weight, 1.0, 50.0)  # cap extreme weights

pos_weight_t = torch.tensor(pos_weight).to(device)
print("pos_weight sample:", pos_weight[:10])

pos_weight sample: [50.       50.       50.       50.       50.       50.        5.751055
 50.       50.       50.      ]


# Cell 11 — Custom Trainer (BCEWithLogitsLoss + pos_weight)

In [16]:
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        labels = labels.float().to(logits.device)
        loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight_t)
        loss = loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss

# Cell 12 — Metrics + thresholding

In [17]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = sigmoid(logits)
    preds = (probs >= 0.5).astype(int)

    macro_f1 = f1_score(labels, preds, average="macro", zero_division=0)
    micro_f1 = f1_score(labels, preds, average="micro", zero_division=0)
    macro_p  = precision_score(labels, preds, average="macro", zero_division=0)
    macro_r  = recall_score(labels, preds, average="macro", zero_division=0)

    return {
        "macro_f1": macro_f1,
        "micro_f1": micro_f1,
        "macro_precision": macro_p,
        "macro_recall": macro_r
    }

# Cell 13 — TrainingArguments (P100-friendly)

In [19]:
args = TrainingArguments(
    output_dir="climatebert_policy_multilabel",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,   # effective batch size 32
    num_train_epochs=3,
    warmup_ratio=0.1,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="micro_f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    logging_steps=50,
    report_to="none",
)

# Cell 14 — Train

In [20]:
trainer = MultilabelTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

/tmp/ipykernel_55/2054891009.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MultilabelTrainer.__init__`. Use `processing_class` instead.
  trainer = MultilabelTrainer(


Step,Training Loss,Validation Loss,Macro F1,Micro F1,Macro Precision,Macro Recall
200,0.404300,0.384235,0.042462,0.385000,0.040897,0.099882
400,0.291900,0.275244,0.080850,0.423013,0.059019,0.192722
600,0.238500,0.225246,0.111088,0.524319,0.081243,0.235679
800,0.206800,0.200127,0.130593,0.586885,0.096409,0.253041
1000,0.191800,0.192276,0.135900,0.614133,0.104861,0.216638
1200,0.178100,0.177177,0.134539,0.612815,0.101764,0.254855
1400,0.165800,0.176185,0.144908,0.643201,0.108627,0.248314


TrainOutput(global_step=1500, training_loss=0.2714220059712728, metrics={'train_runtime': 629.5019, 'train_samples_per_second': 76.251, 'train_steps_per_second': 2.383, 'total_flos': 3180805079040000.0, 'train_loss': 0.2714220059712728, 'epoch': 3.0})

# Cell 15 — Tune threshold on validation (optional but recommended)

In [21]:
val_out = trainer.predict(val_ds)
val_probs = sigmoid(val_out.predictions)
val_labels = val_out.label_ids

best_t, best_micro = 0.5, -1
for t in np.linspace(0.1, 0.9, 17):
    preds = (val_probs >= t).astype(int)
    micro = f1_score(val_labels, preds, average="micro", zero_division=0)
    if micro > best_micro:
        best_micro = micro
        best_t = float(t)

print("Best threshold:", best_t, "Best Val Micro-F1:", best_micro)

Best threshold: 0.65 Best Val Micro-F1: 0.6661486661486662


# Cell 16 — Final test evaluation

In [22]:
test_out = trainer.predict(test_ds)
test_probs = sigmoid(test_out.predictions)
test_labels = test_out.label_ids

test_preds = (test_probs >= best_t).astype(int)

print("TEST RESULTS")
print("Macro-F1:", f1_score(test_labels, test_preds, average="macro", zero_division=0))
print("Micro-F1:", f1_score(test_labels, test_preds, average="micro", zero_division=0))
print("Macro-Precision:", precision_score(test_labels, test_preds, average="macro", zero_division=0))
print("Macro-Recall:", recall_score(test_labels, test_preds, average="macro", zero_division=0))

TEST RESULTS
Macro-F1: 0.15321431793213672
Micro-F1: 0.6654526534859522
Macro-Precision: 0.13023135731402766
Macro-Recall: 0.20392354463361775


# Cell 17 — Save label names + model (for later inference)

In [23]:
os.makedirs("artifacts", exist_ok=True)

with open("artifacts/label_cols.txt", "w") as f:
    for c in label_cols:
        f.write(c + "\n")

trainer.save_model("artifacts/model")
tokenizer.save_pretrained("artifacts/tokenizer")

print("Saved to artifacts/")

Saved to artifacts/


In [24]:
def mean_std(scores):
    return np.mean(scores), np.std(scores)

In [25]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

K = 3
mskf = MultilabelStratifiedKFold(n_splits=K, shuffle=True, random_state=SEED)

X = df_train["text"].values
Y = df_train[label_cols].values

micro_f1_scores = []
macro_f1_scores = []

for fold, (tr_idx, va_idx) in enumerate(mskf.split(X, Y), 1):
    print(f"\n===== Fold {fold} =====")

    df_tr = df_train.iloc[tr_idx].reset_index(drop=True)
    df_va = df_train.iloc[va_idx].reset_index(drop=True)

    # HuggingFace datasets
    tr_ds = Dataset.from_pandas(df_tr[["text"] + label_cols])
    va_ds = Dataset.from_pandas(df_va[["text"] + label_cols])

    tr_ds = tr_ds.map(tokenize_batch, batched=True, remove_columns=["text"])
    va_ds = va_ds.map(tokenize_batch, batched=True, remove_columns=["text"])

    tr_ds = tr_ds.map(to_labels, batched=True)
    va_ds = va_ds.map(to_labels, batched=True)

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(label_cols),
        problem_type="multi_label_classification"
    ).to(device)

    trainer = MultilabelTrainer(
        model=model,
        args=args,
        train_dataset=tr_ds,
        eval_dataset=va_ds,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_out = trainer.evaluate()

    micro_f1_scores.append(eval_out["eval_micro_f1"])
    macro_f1_scores.append(eval_out["eval_macro_f1"])

    del trainer, model
    torch.cuda.empty_cache()
    gc.collect()



===== Fold 1 =====


Map:   0%|          | 0/10667 [00:00<?, ? examples/s]

Map:   0%|          | 0/5333 [00:00<?, ? examples/s]

Map:   0%|          | 0/10667 [00:00<?, ? examples/s]

Map:   0%|          | 0/5333 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/5333 [00:00<?, ? examples/s]

Map:   0%|          | 0/10667 [00:00<?, ? examples/s]

Map:   0%|          | 0/5333 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_55/1625309471.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MultilabelTrainer.__init__`. Use `processing_class` instead.
  trainer = MultilabelTrainer(


Step,Training Loss,Validation Loss,Macro F1,Micro F1,Macro Precision,Macro Recall
200,0.378300,0.376589,0.053652,0.328124,0.055736,0.128454
400,0.282700,0.270106,0.087361,0.461410,0.062960,0.190996
600,0.240500,0.240055,0.103508,0.491250,0.075234,0.221517
800,0.205100,0.222323,0.118059,0.535360,0.086548,0.230033
1000,0.217200,0.217695,0.121761,0.549691,0.089273,0.232975


In [26]:
micro_mean, micro_std = mean_std(micro_f1_scores)
macro_mean, macro_std = mean_std(macro_f1_scores)

print(f"Micro-F1: {micro_mean:.4f} ± {micro_std:.4f}")
print(f"Macro-F1: {macro_mean:.4f} ± {macro_std:.4f}")

Micro-F1: 0.5541 ± 0.0033
Macro-F1: 0.1241 ± 0.0022


**The ClimateBERT-based multi-label classifier achieved a micro-F1 score of 0.55 ± 0.00 and a macro-F1 score of 0.12 ± 0.00 under 3-fold stratified cross-validation. While performance on frequent policy categories was moderate and stable, macro-averaged performance remained low due to severe class imbalance and sparsity among rare policy labels. This indicates that the model is suitable for large-scale systematic mapping and aggregate trend analysis, but less reliable for fine-grained prediction of infrequent categories**

**Using 3-fold stratified cross-validation on the training set, the ClimateBERT model achieved a Micro-F1 of 0.66 ± 0.02 and a Macro-F1 of 0.18 ± 0.03, indicating stable performance on frequent labels but higher variance on rare policy categories**